For this tutorial the following packages are required in addition to the `simframe` requirements:

* `scipy`

In this tutorial you'll learn about the basic usage of `simframe`.

# Simple Integration

This lesson is a showcase for setting up a simple integration with `simframe`. And explains how to write/read data files.

The goal is to integrate the following differential equation

$\frac{\mathrm{d}Y}{\mathrm{d}x} = b Y$

with the initial condition

$Y \left( 0 \right) = A$.

The solution of this problem is given by

$Y \left( x \right) = A \cdot \exp \left( bx \right)$.

## Setting the problem parameters

In [1]:
A  = 10. # Initial value
b  = -1. # decay scale
dx = 0.1 # Stepsize

## Initializing the simulation frame

In [2]:
from simframe import Frame

In [3]:
sim = Frame(description="Simple Integration")

In [4]:
sim

Frame (Simple Integration)
--------------------------
    Integrator   : not specified
    Writer       : not specified

This created an empty framework `sim`.

As you can see the Integrator and Writer of the Frame object are not set initially. The Writer contains the instructions for writing output files. If you do not want to outputs you can ommit setting it.

The Integrator, on the other hand, has to be set for a successful execution. At the minimum it has to control when to stop the simulation.

## Setting the Writer

`simframe` comes with a built-in writer for writing outputs in the `.hdf5` file format using the `h5py` module.

In [5]:
from simframe import writers

The writer can be simply assigned to the frame object

In [6]:
from simframe.io import Writer

In [7]:
isinstance(writers.namespacewriter, Writer)

True

In [8]:
sim.writer = writers.namespacewriter

In [9]:
sim

Frame (Simple Integration)
--------------------------
    Integrator   : not specified
    Writer       : Writer (Namespace writer)

## Adding fields

We now have to add fields to our simulation frame, which has a method for this task.

Here we add the field for our variable `Y` and initialize it with the initial condition `A`.\
The value you set has to be of the correct shape and data type. It cannot be changed later.

In [10]:
sim.addfield("Y", A)

In [11]:
sim

Frame (Simple Integration)
--------------------------
    Y            : Field
  -----
    Integrator   : not specified
    Writer       : Writer (Namespace writer)

The field `Y` can be easily accessed and manipulated. All `NumPy` functions for `ndarray` work on it.

For example:

In [12]:
sim.Y

10.0

In [13]:
sim.Y * 2

20.0

In [14]:
import numpy as np

np.exp(sim.Y)

22026.465794806718

We now have to add a field for `x`. But since this is our integration variable, we have to add a special field for it.\
Here we set the initial value to `0`:

In [15]:
sim.addintegrationvariable("x", 0.)

In [16]:
sim

Frame (Simple Integration)
--------------------------
    x            : Field, Integration variable
    Y            : Field
  -----
    Integrator   : not specified
    Writer       : Writer (Namespace writer)

An integration variable is a special field that has additional functionality for the stepsize of our simulation.

## Adding functions

First we define a function that controls the stepsize of the simulation. It needs to have the frame object as argument. Therefore, anything within the object can be accessed. The function needs to return the value of the stepsize

In this simple integration, we want to have a constant stepsize, that we defined above.

In [17]:
def fdx(sim):
    return dx

We can now simply assign this function to the integration variable

In [18]:
sim.x.updater = fdx

Furthermore, the integration variable needs to know the desired snapshots. Snapshots define the value of the integration variable at which an output should be written. Even if you do not want to write output files the list of snapshots needs to contain at least one value, which defines when the simulation should be stopped.

Here we want to have 20 output files written between `x=0.5` and `x=10.` and assign them to the integration variable.

In [19]:
snaps = np.linspace(0.5, 10., 20)
sim.x.snapshots = snaps

The integration variable has now anything it needs to access its additional functionality which are the stepsize,

In [20]:
sim.x.stepsize

0.1

the maximum possible stepsize, i.e., the stepsize until the next snapshot,

In [21]:
sim.x.maxstepsize

0.5

and the value of the next snapshot, which is here identical to the maximum stepsize.

In [22]:
sim.x.nextsnapshot

0.5

## Setting the differential equation

We also have to define the differential equation of our variable `Y`, that we want to integrate. The function needs as argument the frame object, the itnegration variable, and the variable itself.

In [23]:
def dYdx(sim, x, Y):
    return b*Y

We can simply assign it to the variable as before.

In [24]:
sim.Y.differentiator = dYdx

## Setting the integrator

Last but not least we have to specify the integrator which controls the simulation. The integrator needs the integration variable as argument upon initialization.

In [25]:
from simframe import Integrator

sim.integrator = Integrator(sim.x, description="Simple first order integration")

In [26]:
sim

Frame (Simple Integration)
--------------------------
    x            : Field, Integration variable
    Y            : Field
  -----
    Integrator   : Integrator (Simple first order integration)
    Writer       : Writer (Namespace writer)

If we ran the simulation now, the output files would be written, but `Y` would stay constant. The reason is that the integrator needs a set of instructions that specifiy what to do.

An instruction specifies the integration scheme, the variable to be integrated, and the fraction of the stepsize it should operate on. `simframe` provides a simple Euler 1st order scheme. We'll explain later how to create your own schemes.

In [27]:
from simframe import Instruction
from simframe import schemes

Here we set up our instruction for integrating `Y` with a simple Euler 1st order scheme by using the full timestep.

In [28]:
instructions = [Instruction(schemes.expl_1_euler, sim.Y)]

We simply add this list to our integrator.

In [29]:
sim.integrator.instructions = instructions

## Running the simulation

The simulation is now ready to go!

In [30]:
sim.writer

Writer (Namespace writer)
-------------------------
    Writer to convert Frame objects into namespace.
    Can be accessed with <Writer>.read.output(i) and
    <Writer>.read.all().

    Verbosity      : 1

In [31]:
sim.writer.verbose = 0

In [32]:
sim.run()

Since the data directory did not exist before, it was created by `simframe` and the output files have been written.

## Reading the outputs

Every writer should provide a reader for its specific file format. It can either list all data files in the data directory,

read a single output file,

output3 = sim.writer.read.output(3)

output3

or read the full data set.

data = sim.writer.read.all()

The data is easily accesible from the framework namespace, e.g.

In [33]:
sim.writer.read.all()

namespace(Y=array([1.00000000e+01, 5.90490000e+00, 3.48678440e+00, 2.05891132e+00,
                   1.21576655e+00, 7.17897988e-01, 4.23911583e-01, 2.50315550e-01,
                   1.47808829e-01, 8.72796357e-02, 5.15377521e-02, 3.04325272e-02,
                   1.79701030e-02, 1.06111661e-02, 6.26578748e-03, 3.69988485e-03,
                   2.18474501e-03, 1.29007008e-03, 7.61773480e-04, 4.49819622e-04,
                   2.65613989e-04]),
          x=array([ 0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ,
                    5.5,  6. ,  6.5,  7. ,  7.5,  8. ,  8.5,  9. ,  9.5, 10. ]))

In [34]:
sim.writer.read.output(2)

{'Y': 3.486784400999999, 'x': 1.0}

## Fitting of the data

This is a simple fit of the data to see if the original values of `A` and `b` can be obtained.

You can re-run the simulation and play around with `A`, `b`, and `dx` to see how this simple integration behaves. Note: `dx` will never be larger than the distance to the next snapshot. If you want to increase `dx` beyond that you have to reduce the number of snapshots.

In [35]:
sim.writer._buffer

[{'Y': 10.0, 'x': 0.0},
 {'Y': 5.9049, 'x': 0.5},
 {'Y': 3.486784400999999, 'x': 1.0},
 {'Y': 2.05891132094649, 'x': 1.5},
 {'Y': 1.2157665459056932, 'x': 2.0},
 {'Y': 0.7178979876918532, 'x': 2.5},
 {'Y': 0.42391158275216256, 'x': 3.0},
 {'Y': 0.2503155504993246, 'x': 3.5},
 {'Y': 0.14780882941434623, 'x': 4.0},
 {'Y': 0.08727963568087714, 'x': 4.5},
 {'Y': 0.05153775207320105, 'x': 5.0},
 {'Y': 0.030432527221704433, 'x': 5.5},
 {'Y': 0.017970102999144217, 'x': 6.0},
 {'Y': 0.01061116611996465, 'x': 6.5},
 {'Y': 0.006265787482177915, 'x': 7.0},
 {'Y': 0.00369988485035123, 'x': 7.5},
 {'Y': 0.0021847450052838936, 'x': 8.0},
 {'Y': 0.001290070078170084, 'x': 8.5},
 {'Y': 0.0007617734804586517, 'x': 9.0},
 {'Y': 0.0004498196224760284, 'x': 9.5},
 {'Y': 0.00026561398887586953, 'x': 10.0}]